In [1]:
import sys

sys.path.append("../")  # up one level

import os
import shutil

import gcsfs
import geopandas as gpd
import pandas as pd
import annual_ridership_module

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis.tables import tbls
from segment_speed_utils.project_vars import PUBLIC_GCS
from siuba import _, collect, count, filter, select, show_query
from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/ntd/"

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Work plan

## COMPLETE ~~Need List of all annual reporters~~
- Full, Reduced, Rural reporters all submit an annual Report to NTD
- used `dim_annual_service_agencies`

## COMPLETE ~~Need to assign rural and reduced annual reporters to RTPAs~~ 
- Full reporters are already mapped to RTPAs via the monthly report
- but reduced and rurap reporters need to be mapped to RTPAs
- sjoined a map of CA census designated places to RTPA map, then merged with `dim_annual_service_agenices` to attach RTPA info

## COMPLETE ~~Export processed data to parquet~~
- to be used for aggregation

## COMPLTE ~~Make new ntd_id to RTPA crosswalk~~
- to be used for future analyses and merges

## COMPLETE ~~Import formulas from monthly report for aggregation~~
- modified `produce_annual_ntd_ridership_data_by_rtpa`

## Test using data from new warehouse table `fct_service_data_and_operating_expenses_time_series_by_mode_upt`
- https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.fct_service_data_and_operating_expenses_time_series_by_mode_upt
---

## read in `fct_service_data_and_operating_expenses_time_series_by_mode_upt`

In [2]:
# new warehouse table
# includes upt, mode, tos and report year starting from 2018
# contains historical,inactive agencies

get_ntd_time_series = (
    tbls.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_upt()
    >> filter(_.state.str.contains("CA") | 
              _.state.str.contains("NV"), # to get lake Tahoe Transportation back
              _.year >= "2018",
              _.city != None,
              _.primary_uza_name.str.contains(", CA") | 
              _.primary_uza_name.str.contains("CA-NV") |
              _.primary_uza_name.str.contains("California Non-UZA") | 
              _.primary_uza_name.str.contains("El Paso, TX--NM") # something about Paso 
             )
    >> select(
        'agency_name',
        'agency_status',
        'city',
        'legacy_ntd_id',
        'mode',
        'ntd_id',
        'reporter_type',
        'reporting_module',
        'service',
        'state',
        'uace_code',
        'primary_uza_name',
        'uza_population',
        'year',
        'upt',
    )
    >> collect()
)

#what does group by look like
ntd_time_series = get_ntd_time_series.groupby(
    [
        "agency_name",
        'agency_status',
        "city",
        "state",
        "ntd_id",
        'primary_uza_name',
        "reporter_type",
        "mode",
        "service",
        "year"
    ]
).agg({
    "upt":"sum"
}).sort_values(by="ntd_id").reset_index()

display(
    get_ntd_time_series.info(),
    ntd_time_series.info(),
    ntd_time_series["state"].value_counts(),
    ntd_time_series["primary_uza_name"].value_counts()
    
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4050 entries, 0 to 4049
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   agency_name       4050 non-null   object 
 1   agency_status     4050 non-null   object 
 2   city              4050 non-null   object 
 3   legacy_ntd_id     3324 non-null   object 
 4   mode              4050 non-null   object 
 5   ntd_id            4038 non-null   object 
 6   reporter_type     4050 non-null   object 
 7   reporting_module  3768 non-null   object 
 8   service           4050 non-null   object 
 9   state             4050 non-null   object 
 10  uace_code         4050 non-null   int64  
 11  primary_uza_name  4050 non-null   object 
 12  uza_population    4050 non-null   int64  
 13  year              4050 non-null   object 
 14  upt               2642 non-null   float64
dtypes: float64(1), int64(2), object(12)
memory usage: 474.7+ KB
<class 'pandas.core.frame.Dat

None

None

CA    3996
NV      42
Name: state, dtype: int64

Los Angeles--Long Beach--Anaheim, CA               1116
California Non-UZA                                  876
San Francisco--Oakland, CA                          258
Sacramento, CA                                      240
San Diego, CA                                       156
Modesto, CA                                          90
Riverside--San Bernardino, CA                        90
Stockton, CA                                         78
San Jose, CA                                         60
Visalia, CA                                          54
Santa Rosa, CA                                       48
Vallejo, CA                                          48
Thousand Oaks, CA                                    42
San Luis Obispo, CA                                  42
Lake Tahoe, CA-NV                                    42
Santa Barbara, CA                                    42
Oxnard--San Buenaventura (Ventura), CA               42
Madera, CA                                      

In [3]:
ntd_time_series[~ntd_time_series["primary_uza_name"].str.contains(", CA")]["primary_uza_name"].value_counts()
# who has uza in texas new mex?!

California Non-UZA    876
El Paso, TX--NM        12
Name: primary_uza_name, dtype: int64

In [4]:
ntd_time_series[ntd_time_series["primary_uza_name"].str.contains("El Paso, TX--NM")] # just 1 agency, PAso Robles Transit Services

,agency_name,agency_status,city,state,ntd_id,primary_uza_name,reporter_type,mode,service,year,upt
1872,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,DR,PT,2020,0.0
1873,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,MB,PT,2022,0.0
1874,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,DR,PT,2021,0.0
1875,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,MB,PT,2020,0.0
1876,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,MB,PT,2019,0.0
1877,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,MB,PT,2021,0.0
1878,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,DR,PT,2023,0.0
1879,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,DR,PT,2022,0.0
1880,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,MB,PT,2023,0.0
1881,Paso Robles Transit Services (PE),Inactive,Paso Robles,CA,90195,"El Paso, TX--NM",Reduced Reporter,DR,PT,2018,0.0


In [5]:
# how many cities in ntd_time_serires are there?
# wonder if all the cities will be in the crosswalk
ntd_time_series["city"].nunique() # 183. should be enough


184

## See `explore_rtpa_ntd_crosswalk` notebook to see how the new rtpa to ntd id crosswalk was made

## read in crosswalk from gcs

In [6]:
test_data = pd.read_parquet(f"{GCS_FILE_PATH}annual_report_data_2022-2023.parquet")
rtpa_ntd_crosswalk = pd.read_parquet(f"{GCS_FILE_PATH}ntd_id_rtpa_crosswalk_all_reporter_types.parquet")

display(
    test_data.head(),
    rtpa_ntd_crosswalk.head()
)

,report_year,ntd_id,agency,reporter_type,organization_type,city,state,primary_uza_name,actual_vehicles_passenger_car_revenue_hours,actual_vehicles_passenger_car_revenue_miles,unlinked_passenger_trips_upt,cdp_name,RTPA,_merge
0,2022,90003,San Francisco Bay Area Rapid Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Oakland,CA,"San Francisco--Oakland, CA",2486029.0,78554914.0,38224072.0,Oakland,Metropolitan Transportation Commission,both
1,2023,90003,"San Francisco Bay Area Rapid Transit District,...",Full Reporter,Independent Public Agency or Authority of Tran...,Oakland,CA,"San Francisco--Oakland, CA",2724074.0,85233749.0,50764402.0,Oakland,Metropolitan Transportation Commission,both
2,2022,90004,Golden Empire Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Bakersfield,CA,"Bakersfield, CA",292325.0,3758507.0,3201046.0,Bakersfield,Kern Council of Governments,both
3,2023,90004,Golden Empire Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Bakersfield,CA,"Bakersfield, CA",289338.0,3924016.0,3293593.0,Bakersfield,Kern Council of Governments,both
4,2023,90006,Santa Cruz Metropolitan Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Santa Cruz,CA,"Santa Cruz, CA",214748.0,2975126.0,3350026.0,Santa Cruz,Santa Cruz County Regional Transportation Comm...,both


,ntd_id,agency_name,reporter_type,agency_status,city,state,RTPA
0,90003,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission
1,90004,Golden Empire Transit District (GET),Full Reporter,Active,Bakersfield,CA,Kern Council of Governments
2,90006,Santa Cruz Metropolitan Transit District (SCMTD),Full Reporter,Active,Santa Cruz,CA,Santa Cruz County Regional Transportation Comm...
3,90007,City of Modesto (MAX),Full Reporter,Inactive,Modesto,CA,Stanislaus Council of Governments
4,90008,City of Santa Monica (BBB) - Department of Tra...,Full Reporter,Active,Santa Monica,CA,Southern California Association of Governments


## test merge of `ntd_time_series` with `rtpa ntd crosswalk`

In [7]:
ntd_time_series_rtpa = ntd_time_series.merge(
    rtpa_ntd_crosswalk,
    on="ntd_id",
    how="left",
    indicator=True
)
display(
    ntd_time_series_rtpa.info(),
    ntd_time_series_rtpa["_merge"].value_counts()
)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4038 entries, 0 to 4037
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   agency_name_x     4038 non-null   object  
 1   agency_status_x   4038 non-null   object  
 2   city_x            4038 non-null   object  
 3   state_x           4038 non-null   object  
 4   ntd_id            4038 non-null   object  
 5   primary_uza_name  4038 non-null   object  
 6   reporter_type_x   4038 non-null   object  
 7   mode              4038 non-null   object  
 8   service           4038 non-null   object  
 9   year              4038 non-null   object  
 10  upt               4038 non-null   float64 
 11  agency_name_y     4038 non-null   object  
 12  reporter_type_y   4038 non-null   object  
 13  agency_status_y   4038 non-null   object  
 14  city_y            4038 non-null   object  
 15  state_y           4038 non-null   object  
 16  RTPA              4038 n

None

both          4038
left_only        0
right_only       0
Name: _merge, dtype: int64

In [8]:
# gtg
ntd_time_series_rtpa.head()

,agency_name_x,agency_status_x,city_x,state_x,ntd_id,primary_uza_name,reporter_type_x,mode,service,year,upt,agency_name_y,reporter_type_y,agency_status_y,city_y,state_y,RTPA,_merge
0,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,MB,PT,2018,0.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
1,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2018,127874512.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
2,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2019,125105460.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
3,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2020,88698878.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
4,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2021,17125273.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both


## Testing the `add_change_column` from module
- use on `ntd_time_series_rtpa` to add new column
1. previous UPT from prev year/ntd_id/mode/tos
2. change upt % from prev year/ntd_/mode/tos


In [9]:
sort_cols2 =  ["ntd_id",
               "year", 
               "service",
               "mode"
              ] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months
group_cols2 = ["ntd_id",
               "mode",
               "service"
              ]
    
#df[["period_year","period_month"]] = df[["period_year","period_month"]].astype(int)
#ntd_time_series_rtpa.sort_values(sort_cols2).groupby(group_cols2)["upt"].apply(lambda x: x.shift(1))

In [10]:
ntd_time_series_rtpa.sort_values(sort_cols2).head()

,agency_name_x,agency_status_x,city_x,state_x,ntd_id,primary_uza_name,reporter_type_x,mode,service,year,upt,agency_name_y,reporter_type_y,agency_status_y,city_y,state_y,RTPA,_merge
1,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2018,127874512.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
20,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,YR,DO,2018,207580.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
28,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,DR,PT,2018,0.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
0,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,MB,PT,2018,0.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both
13,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,MG,PT,2018,962251.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both


In [11]:
ntd_time_series_rtpa.sort_values(sort_cols2).groupby(group_cols2)["upt"].apply(lambda x: x.shift(1)).head()

/tmp/ipykernel_759/3290079175.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  ntd_time_series_rtpa.sort_values(sort_cols2).groupby(group_cols2)["upt"].apply(lambda x: x.shift(1)).head()


1    NaN
20   NaN
28   NaN
0    NaN
13   NaN
Name: upt, dtype: float64

In [12]:
test = ntd_time_series_rtpa.assign(
        previous_y_upt = (ntd_time_series_rtpa.sort_values(sort_cols2)
                        .groupby(group_cols2)["upt"] 
                        .apply(lambda x: x.shift(1))
                       )
    )

/tmp/ipykernel_759/1420364381.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1))


In [13]:
test[test["ntd_id"]=="90253"].sort_values(by=["year","mode","service"])

,agency_name_x,agency_status_x,city_x,state_x,ntd_id,primary_uza_name,reporter_type_x,mode,service,year,upt,agency_name_y,reporter_type_y,agency_status_y,city_y,state_y,RTPA,_merge,previous_y_upt
2462,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2018,36975.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,NaN
2459,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2018,140383.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,NaN
2463,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2019,32402.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,36975.0
2465,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2019,131018.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,140383.0
2464,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2020,17136.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,32402.0
2457,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2020,116995.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,131018.0
2461,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2021,15372.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,17136.0
2456,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2021,58286.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,116995.0
2460,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2022,14633.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,15372.0
2455,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2022,105812.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,58286.0


In [14]:
ntd_rtpa_change_cols = annual_ridership_module.add_change_columns(ntd_time_series_rtpa)

/home/jovyan/data-analyses/ntd/annual_ridership_report/annual_ridership_module.py:46: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1))


In [15]:
ntd_rtpa_change_cols[ntd_rtpa_change_cols["ntd_id"]=="90253"].sort_values(by=["year","service","mode"])

,agency_name_x,agency_status_x,city_x,state_x,ntd_id,primary_uza_name,reporter_type_x,mode,service,year,upt,agency_name_y,reporter_type_y,agency_status_y,city_y,state_y,RTPA,_merge,previous_y_upt,change_1yr,pct_change_1yr
2462,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2018,36975.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,NaN,NaN,NaN
2459,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2018,140383.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,NaN,NaN,NaN
2463,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2019,32402.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,36975.0,-4573.0,-0.1411
2465,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2019,131018.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,140383.0,-9365.0,-0.0715
2464,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2020,17136.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,32402.0,-15266.0,-0.8909
2457,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2020,116995.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,131018.0,-14023.0,-0.1199
2461,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2021,15372.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,17136.0,-1764.0,-0.1148
2456,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2021,58286.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,116995.0,-58709.0,-1.0073
2460,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,DR,PT,2022,14633.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,15372.0,-739.0,-0.0505
2455,City of Bell Gardens - public works,Active,Bell,CA,90253,"Los Angeles--Long Beach--Anaheim, CA",Reduced Reporter,MB,PT,2022,105812.0,City of Bell Gardens - public works,Reduced Reporter,Active,Bell,CA,Southern California Association of Governments,both,58286.0,47526.0,0.4492


## Full Test `produce_annual_ntd_ridership_data_by_rtpa` function
- need to updat to new NTD data table.


In [16]:
check_id=["90253","90227","90259","90286"]

display(
    ntd_time_series_rtpa[ntd_time_series_rtpa["ntd_id"].isin(check_id)]["city_x"].unique(),
    rtpa_ntd_crosswalk[rtpa_ntd_crosswalk["ntd_id"].isin(check_id)]["city"].unique()
)

# cities match, no need for the dictionary update anymore in the function

array(['Moorpark', 'Bell', 'Cerritos', 'Montery Park'], dtype=object)

array(['Moorpark', 'Bell', 'Cerritos', 'Montery Park'], dtype=object)

In [17]:
def produce_annual_ntd_ridership_data_by_rtpa():
    """
    Function that ingest ridership data from `dim_annual_service_agencies`, filters for CA agencies.
    Merges in ntd_id_to_RTPA_crosswalk.
    
    """
    from annual_ridership_module import add_change_columns
    
    print("ingest annual ridership data from warehouse")
    
    ntd_service = (
    tbls.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_upt()
    >> filter(_.state.str.contains("CA") | 
              _.state.str.contains("NV"), # to get lake Tahoe Transportation back
              _.year >= "2018",
              _.city != None,
              _.primary_uza_name.str.contains(", CA") | 
              _.primary_uza_name.str.contains("CA-NV") |
              _.primary_uza_name.str.contains("California Non-UZA") | 
              _.primary_uza_name.str.contains("El Paso, TX--NM") # something about Paso 
             )
    >> select(
        'agency_name',
        'agency_status',
        'city',
        'legacy_ntd_id',
        'mode',
        'ntd_id',
        'reporter_type',
        'reporting_module',
        'service',
        'state',
        'uace_code',
        'primary_uza_name',
        'uza_population',
        'year',
        'upt',
    )
    >> collect())
    
    ntd_service = ntd_service.groupby(
        [
            "agency_name",
            'agency_status',
            "city",
            "state",
            "ntd_id",
            'primary_uza_name',
            "reporter_type",
            "mode",
            "service",
            "year"
        ]
    ).agg({
        "upt":"sum"
    }).sort_values(by="ntd_id").reset_index()
        
    #print("clean up warehouse data")
    
    #update_dict={
    #    "90227":"Moorpark",
    #    "90253":"Bell Gardens",
    #    "90259":"Cerritos",
    #    "90286":"Monterey Park",
    #}
    
    #for i, v in update_dict.items():
    #    ntd_service.loc[ntd_service["ntd_id"]== i,"city"] = v
    
    print("read in new `ntd_id_to_rtpa_all_reporter_types` crosswalk") 
    
    ntd_to_rtpa_crosswalk = pd.read_parquet(f"{GCS_FILE_PATH}ntd_id_rtpa_crosswalk_all_reporter_types.parquet")
        
    print("merge ntd data to crosswalk")
    
    ntd_data_by_rtpa = ntd_service.merge(
    ntd_to_rtpa_crosswalk,
    how="left",
    on=[
        "ntd_id",
        #"agency", # sometime agency name change, but ntd id stays the same. causing unmerged rows
        #"reporter_type",
        #"city"
    ],
    indicator=True
    ).rename(
    columns={
       "actual_vehicles_passenger_car_revenue_hours":"vrh",
       "actual_vehicles_passenger_car_revenue_miles":"vrm",
       "unlinked_passenger_trips_upt":"upt" 
    }
    )
    
    print(ntd_data_by_rtpa._merge.value_counts())
        
    if len(ntd_data_by_rtpa[ntd_data_by_rtpa._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
    
    print("add `change_column` to data")
    ntd_data_by_rtpa = add_change_columns(ntd_data_by_rtpa)
    
    return ntd_data_by_rtpa

# will add step to export data to gcs in final script

In [18]:
annual_ridership_report_data = produce_annual_ntd_ridership_data_by_rtpa()


ingest annual ridership data from warehouse
read in new `ntd_id_to_rtpa_all_reporter_types` crosswalk
merge ntd data to crosswalk
both          4038
left_only        0
right_only       0
Name: _merge, dtype: int64
add `change_column` to data


/home/jovyan/data-analyses/ntd/annual_ridership_report/annual_ridership_module.py:46: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1))


In [19]:
if annual_ridership_report_data.shape == ntd_time_series_rtpa.shape:
    pass
else: print(f"""
{annual_ridership_report_data.shape} vs {ntd_time_series_rtpa.shape}.
{annual_ridership_report_data.columns}
{ntd_time_series_rtpa.columns}
""")

# gtg, ntd_time_series_rtpa doesnt apply the change 1 year calculations, but the lengths are the same 


(4038, 21) vs (4038, 18).
Index(['agency_name_x', 'agency_status_x', 'city_x', 'state_x', 'ntd_id',
       'primary_uza_name', 'reporter_type_x', 'mode', 'service', 'year', 'upt',
       'agency_name_y', 'reporter_type_y', 'agency_status_y', 'city_y',
       'state_y', 'RTPA', '_merge', 'previous_y_upt', 'change_1yr',
       'pct_change_1yr'],
      dtype='object')
Index(['agency_name_x', 'agency_status_x', 'city_x', 'state_x', 'ntd_id',
       'primary_uza_name', 'reporter_type_x', 'mode', 'service', 'year', 'upt',
       'agency_name_y', 'reporter_type_y', 'agency_status_y', 'city_y',
       'state_y', 'RTPA', '_merge'],
      dtype='object')



In [20]:
annual_ridership_report_data.head()

,agency_name_x,agency_status_x,city_x,state_x,ntd_id,primary_uza_name,reporter_type_x,mode,service,year,upt,agency_name_y,reporter_type_y,agency_status_y,city_y,state_y,RTPA,_merge,previous_y_upt,change_1yr,pct_change_1yr
0,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,MB,PT,2018,0.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both,NaN,NaN,NaN
1,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2018,127874512.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both,NaN,NaN,NaN
2,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2019,125105460.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both,127874512.0,-2769052.0,-0.0221
3,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2020,88698878.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both,125105460.0,-36406582.0,-0.4105
4,San Francisco Bay Area Rapid Transit District ...,Active,Oakland,CA,90003,"San Francisco--Oakland, CA",Full Reporter,HR,DO,2021,17125273.0,San Francisco Bay Area Rapid Transit District ...,Full Reporter,Active,Oakland,CA,Metropolitan Transportation Commission,both,88698878.0,-71573605.0,-4.1794


### export report data to GCS, as parquet,
- to be read in by portfolio

In [21]:
# annual_ridership_report_data.to_parquet(f"{GCS_FILE_PATH}annual_ridership_report_data.parquet")

## read in data from GCS to make sure things work 

In [22]:
annual_ridership_report_data= pd.read_parquet(f"{GCS_FILE_PATH}annual_ridership_report_data.parquet")

In [23]:
annual_ridership_report_data['RTPA'].value_counts()

Southern California Association of Governments          194
Metropolitan Transportation Commission                   50
Kern Council of Governments                              21
San Joaquin Council of Governments                       14
Sacramento Area Council of Governments                   14
Santa Barbara County Association of Governments          14
Humboldt County Association of Governments                8
Madera County Transportation Commission                   8
Placer County Transportation Planning Agency              6
San Diego Association of Governments                      6
Tulare County Association of Governments                  6
San Luis Obispo Council of Governments                    6
Kings County Association of Governments                   6
Stanislaus Council of Governments                         5
Del Norte Local Transportation Commission                 5
Merced County Association of Governments                  4
Fresno Council of Governments           